In [1]:
import os
import time 
import datetime

def print_table(data: dict[str, list[float]], precision: int = 4):
    # Convert to list of keys and find number of rows
    headers = list(data.keys())
    headers = ["index"] + headers        
    n_rows = len(next(iter(data.values())))
    data["index"] = list(range(1, n_rows+1))

    # Format floats to desired precision
    fmt = f"{{:.{precision}f}}"

    # Compute column widths
    col_widths = []
    for h in headers:
        values = data[h]
        max_val_len = max(len(fmt.format(v)) for v in values)
        col_widths.append(max(len(h), max_val_len))

    # Build header row
    header_row = " | ".join(f"{h:>{w}}" for h, w in zip(headers, col_widths))
    separator = "-+-".join("-" * w for w in col_widths)

    # Print table
    print(header_row)
    print(separator)
    for i in range(n_rows):
        row = " | ".join(fmt.format(data[h][i]).rjust(w) for h, w in zip(headers, col_widths))
        print(row)

In [2]:
def dt_to_intkey(date_str: str, time_str: str) -> float:
    dt = datetime.datetime.strptime(f"{date_str} {time_str}", "%Y_%m_%d %H_%M_%S")
    return dt.timestamp()

In [6]:
data_dir = "../../logs_cluster/logs/full_runs/16_12_2025_ycocg_arm_smol_no_colreg_gain_test_multiimarm_kodak/results/"
res_files_unfiltered = [f for f in os.listdir(data_dir) if f.endswith(".log")]
res_files_unfiltered.sort()
# print(*res_files_unfiltered, sep="\n")
dates, times = zip(*(s.split("__")[:2] for s in res_files_unfiltered))
creation_times = {
    # key is date_time string converted to int/float for sorting
    index: dt_to_intkey(date, time)
    for index, (date, time) in enumerate(zip(dates, times))
}
only_after = dt_to_intkey("2025_10_30", "00_00_00")
res_files = [
    res_files_unfiltered[index]
    for index, creation_time in creation_times.items()
    if creation_time >= only_after
]
# print(*res_files, sep="\n")
print(len(res_files), "files found after filtering.")

24 files found after filtering.


In [7]:
# Example log lines we are looking for:
# Using color space YCoCg with bitdepths [8, 9, 9]
# Using image ARM: True
# Using encoder gain: 64
# Using multi-region image ARM: True
# Using color regression: False
# Total training iterations: 142200
# Total MAC per pixel: 1694.3125
# Final results after quantization: Loss: 3.278064727783203, Rate NN: 0.11986033121744792, Rate Latent: 0.1637668013572693, Rate Img: 2.9944374561309814
# Rate Img bistream: 7.751302083333333

using_color_space = ""
using_image_arm = False
using_encoder_gain = 0
using_multi_region_image_arm = False
using_color_regression = False
total_training_iterations = 0
total_mac_per_pixel = 0.0
final_results = []
rate_img_bistream = []

# with open(os.path.join(data_dir, res_files[0]), "r") as infile:
#     lines = infile.readlines()    
#     for line in lines:
#         if line.startswith("Using color space"):
#             using_color_space = line[len("Using color space") :].strip().split(" with bitdepths")[0]
#         if line.startswith("Using image ARM:"):
#             using_image_arm = line[len("Using image ARM:"):].strip() == "True"
#         if line.startswith("Using encoder gain:"):
#             using_encoder_gain = int(line[len("Using encoder gain:"):].strip())
#         if line.startswith("Using multi-region image ARM:"):
#             using_multi_region_image_arm = line[len("Using multi-region image ARM:"):].strip() == "True"
#         if line.startswith("Using color regression:"):
#             using_color_regression = line[len("Using color regression:"):].strip() == "True"
#         if line.startswith("Total training iterations:"):
#             total_training_iterations = int(line[len("Total training iterations:"):].strip())
#         if line.startswith("Total MAC per pixel:"):
#             total_mac_per_pixel = float(line[len("Total MAC per pixel:"):].strip())

#     if using_color_space == "":
#         print(f"Could not determine color space for file {res_files[0]}, skipping.")
#         raise ValueError("No color space found.")
# print("Using color space:", using_color_space)
# print("Using Image ARM:", using_image_arm)
# print("Using encoder gain:", using_encoder_gain)
# print("Using multi-region image ARM:", using_multi_region_image_arm)
# print("Using color regression:", using_color_regression)
# print("Total training iterations:", total_training_iterations)
# print("Total MAC per pixel:", total_mac_per_pixel)


for f in res_files:
    with open(os.path.join(data_dir, f), "r") as infile:
        lines = infile.readlines()   

        for line in lines:
            if line.startswith("Rate Img bistream:"):
                rate_img_bistream.append(float(line[len("Rate Img bistream:"):].strip()))
            if "Final results after quantization" in line:
                parts = line[len("Final results after quantization:") :].strip().split(", ")
                im_index = int(f.split("_")[-1][len("kodim") :][: -len(".log")]) - 1
                final_results.append({})
                for part in parts:
                    key, value = part.split(": ")
                    final_results[-1][key] = float(value)

# print("Rate Img bistreams:", rate_img_bistream)
# print("Final results collected:", final_results)

def list_dict_to_dict_list(
    lst: list[dict[str, float]],
) -> dict[str, list[float]]:
    if not lst:
        return {}
    keys = lst[0].keys()
    dict_list = {key: [] for key in keys}
    for d in lst:
        for key in keys:
            dict_list[key].append(d.get(key, 0.0))
    return dict_list

results_table = list_dict_to_dict_list(final_results)
# results_table["Rate Img bistream"] = rate_img_bistream
print_table(results_table)

for key in results_table.keys():
    # results_for_key = [d[0] for d in results_table[key]]
    # results_for_key = list_dict_to_dict_list(results_for_key)

    # print(f"{key}, Using Image ARM: {using_image_arm} Results:")
    # for i in range(len(results_for_key['Loss'])):
    #     results_for_key['Loss'][i] = results_for_key['Loss'][i] - results_for_key['Rate NN'][i]
    #     results_for_key['Rate NN'][i] = results_for_key['Rate NN'][i] #/ (512 * 768 * 3)
    #     results_for_key['Loss'][i] = results_for_key['Loss'][i] + results_for_key['Rate NN'][i]
    # print_table(results_for_key)
    # for key, values in results_for_key.items():
    avg = sum(results_table[key]) / len(results_table[key]) if results_table[key] else 0
    print(f"{key}: {avg:.4f}")

  index |   Loss | Rate NN | Rate Latent | Rate Img
--------+--------+---------+-------------+---------
 1.0000 | 3.2983 |  0.0933 |      0.2003 |   3.0046
 2.0000 | 2.9270 |  0.0933 |      0.3930 |   2.4407
 3.0000 | 2.5626 |  0.0929 |      0.0540 |   2.4157
 4.0000 | 2.8918 |  0.0934 |      0.1841 |   2.6143
 5.0000 | 3.4822 |  0.0928 |      0.2785 |   3.1109
 6.0000 | 3.0644 |  0.0932 |      0.1661 |   2.8052
 7.0000 | 2.7020 |  0.0929 |      0.1352 |   2.4740
 8.0000 | 3.6692 |  0.0928 |      0.1183 |   3.4580
 9.0000 | 2.8052 |  0.0931 |      0.2032 |   2.5089
10.0000 | 2.9322 |  0.0931 |      0.0456 |   2.7936
11.0000 | 2.9722 |  0.0929 |      0.2108 |   2.6685
12.0000 | 2.7733 |  0.0939 |      0.2564 |   2.4230
13.0000 | 3.7052 |  0.0935 |      0.3501 |   3.2615
14.0000 | 3.2473 |  0.0934 |      0.1979 |   2.9561
15.0000 | 2.8384 |  0.0930 |      0.0536 |   2.6918
16.0000 | 2.7455 |  0.0923 |      0.1533 |   2.4998
17.0000 | 2.8151 |  0.0930 |      0.2198 |   2.5023
18.0000 | 3.